In [8]:
# General Imports
import numpy as np
import pandas as pd


# Visualisation Imports
import matplotlib.pyplot as plt

# Scikit Imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_breast_cancer

# Qiskit Imports
from qiskit import Aer, execute
from qiskit import BasicAer
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.ibmq import IBMQ
from qiskit_machine_learning.algorithms import QSVC
from qiskit.utils import QuantumInstance



In [10]:
from qiskit_ibm_provider import IBMQ
provider = IBMQ.load_account()
sim = provider.backend.ibmq_qasm_simulator

Traceback (most recent call last):
  Cell In[10], line 1
    from qiskit_ibm_provider import IBMQ
ImportError: cannot import name 'IBMQ' from 'qiskit_ibm_provider' (/opt/conda/lib/python3.10/site-packages/qiskit_ibm_provider/__init__.py)

Use %tb to get the full traceback.


In [3]:
df = load_breast_cancer()
name = 'breast_cancer'

In [4]:
#Breast Cancer Dataset

data = pd.DataFrame(data=df, columns=df.feature_names)
data.columns = ['feature{}'.format(i) for i in range(0,data.shape[1])]

dataset = pd.DataFrame(df.data).assign(target=df.target)
print (dataset.shape,pd.DataFrame(df.target).nunique().tolist()[-1:])

(569, 31) [2]


In [5]:
# Split dataset
sample_train, sample_test, label_train, label_test = train_test_split(
     df.data, df.target, test_size=0.2, random_state=22)

# Reduce dimensions (the dataset now has a dimensionality of 31 attributes)
n_dim = 30 #number of qubits we want to use
pca = PCA(n_components=n_dim).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# Normalise
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# Scale
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Select
train_size = 100
sample_train = sample_train[:train_size]
label_train = label_train[:train_size]

test_size = 20
sample_test = sample_test[:test_size]
label_test = label_test[:test_size]

With our training and testing datasets ready, we set up the QuantumKernel class with the ZZFeatureMap, and use the BasicAer statevector_simulator to estimate the training and testing kernel matrices.

In [6]:
ibm_quantum_service = QiskitRuntimeService(channel="ibm_quantum")
ibm_quantum_service.backends(min_num_qubits=30)

Traceback (most recent call last):
  Cell In[6], line 1
    ibm_quantum_service = QiskitRuntimeService(channel="ibm_quantum")
NameError: name 'QiskitRuntimeService' is not defined

Use %tb to get the full traceback.


We then simulate the circuit. We will use the `qasm_simulator` since the circuit contains measurements, but increase the number of shots to reduce the effect of sampling noise.

In [7]:
#applying QSVM on simulator
seed = 10598
num_qubits = 30
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement='linear', insert_barriers=True)
#SecondOrderExpansion(feature_dimension=num_qubits, depth=2, entanglement='linear')
zz_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=QuantumInstance(backend = Aer.get_backend("simulator_statevector"), shots=1024, seed_simulator=seed, seed_transpiler=seed))

qsvc = QSVC(quantum_kernel=zz_kernel)

qsvc.fit(sample_train, label_train)

qsvc_score = qsvc.score(sample_test, label_test)

print(f"QSVC classification test score: {qsvc_score}")


Traceback (most recent call last):
  Cell In[7], line 6
    zz_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=QuantumInstance(backend = Aer.get_backend("simulator_statevector"), shots=1024, seed_simulator=seed, seed_transpiler=seed))
  File /opt/conda/lib/python3.10/site-packages/qiskit_aer/aerprovider.py:72 in get_backend
    return super().get_backend(name=name, **kwargs)
  File /opt/conda/lib/python3.10/site-packages/qiskit/providers/provider.py:55 in get_backend
    raise QiskitBackendNotFoundError("No backend matches the criteria")
QiskitBackendNotFoundError: 'No backend matches the criteria'

Use %tb to get the full traceback.


In [ ]:
backend = service.backend("ibmq_qasm_simulator")
job = execute(zz_circuit, backend, shots=8192, 
              seed_simulator=1024, seed_transpiler=1024)
counts = job.result().get_counts(zz_circuit)

This process is then repeated for each pair of training data samples to fill in the training kernel matrix, and between each training and testing data sample to fill in the testing kernel matrix. Note that each matrix is symmetric, so to reduce computation time, only half the entries are calculated explictly. 

In [ ]:
matrix_train = zz_kernel.evaluate(x_vec=sample_train)
matrix_test = zz_kernel.evaluate(x_vec=sample_test, y_vec=sample_train)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train),
              interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test),
              interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("testing kernel matrix")
plt.show()

In [ ]:
zzcb_svc = SVC(kernel=zz_kernel.evaluate)
zzcb_svc.fit(sample_train, label_train)
zzcb_score = zzcb_svc.score(sample_test, label_test)

print(f'Callable kernel classification test score: {zzcb_score}')